In [ ]:
class QuestionNode:
    def __init__(self, question, answer=None):
        self.question = question
        self.answer = answer
        self.children = []  # Child questions that follow from this question/answer
        self.parent = None  # Reference to parent question
    
    def add_child(self, child_node):
        """Add a child question node to this node"""
        self.children.append(child_node)
        child_node.parent = self
        return child_node
    
    def find_node(self, question):
        """Find a node with the given question text"""
        if self.question == question:
            return self
            
        for child in self.children:
            result = child.find_node(question)
            if result:
                return result
                
        return None
    
    def __str__(self):
        return f"Q: {self.question}\nA: {self.answer}"
    
    def print_tree(self, indent=0):
        """Print the tree structure starting from this node"""
        print("  " * indent + f"Q: {self.question}")
        if self.answer:
            print("  " * indent + f"A: {self.answer}")
        
        for child in self.children:
            child.print_tree(indent + 1)


class QuestionTree:
    def __init__(self):
        self.root = None
        self.current_node = None
        self.context_history = []  # Track the conversation path
    
    def add_root_question(self, question, answer=None):
        """Add the first question to create the root of the tree"""
        self.root = QuestionNode(question, answer)
        self.current_node = self.root
        self.context_history = [self.root]
        return self.root
    
    def add_follow_up_question(self, question, answer=None, parent_question=None):
        """Add a follow-up question to a specific parent or to the current question"""
        if not self.root:
            return self.add_root_question(question, answer)
        
        # If parent specified, find that node
        if parent_question:
            parent_node = self.root.find_node(parent_question)
            if not parent_node:
                raise ValueError(f"Parent question '{parent_question}' not found in tree")
        else:
            # Use current conversation node as parent
            parent_node = self.current_node
        
        # Create and add new question node
        new_node = QuestionNode(question, answer)
        parent_node.add_child(new_node)
        
        # Update current node and history
        self.current_node = new_node
        self.context_history.append(new_node)
        
        return new_node
    
    def get_current_context(self):
        """Return the current conversation path"""
        return [(node.question, node.answer) for node in self.context_history]
    
    def navigate_to_question(self, question):
        """Navigate to a specific question in the tree"""
        node = self.root.find_node(question)
        if node:
            self.current_node = node
            
            # Rebuild context history
            history = []
            current = node
            while current:
                history.insert(0, current)
                current = current.parent
            
            self.context_history = history
            return True
        return False
    
    def print_full_tree(self):
        """Print the entire question tree"""
        if self.root:
            self.root.print_tree()
        else:
            print("Tree is empty")
            
    def generate_dot_graph(self):
        """Generate a DOT representation for visualization with Graphviz"""
        if not self.root:
            return "digraph G {}"
        
        dot = ["digraph G {"]
        dot.append('  node [shape=box style="rounded,filled" fillcolor=lightblue];')
        
        # Queue for BFS traversal
        queue = [(self.root, None)]
        node_id = 0
        node_ids = {self.root: node_id}
        
        while queue:
            node, parent_id = queue.pop(0)
            node_text = f"{node.question}\\n{node.answer if node.answer else ''}"
            node_text = node_text.replace('"', '\\"')
            
            dot.append(f'  node{node_ids[node]} [label="{node_text}"];')
            
            if parent_id is not None:
                dot.append(f'  node{parent_id} -> node{node_ids[node]};')
            
            for child in node.children:
                node_id += 1
                node_ids[child] = node_id
                queue.append((child, node_ids[node]))
        
        dot.append("}")
        return "\n".join(dot)


# Example usage based on your conversation:
def demo_conversation_tree():
    # Create the tree
    tree = QuestionTree()
    
    # Add initial question
    root = tree.add_root_question("what is SD?", 
                                 "Software Development involves creating different types: web based, android, desktop, etc.")
    
    # Add follow-up about web-based
    web_node = tree.add_follow_up_question("what is a web based software?", 
                                          "Web-based software runs in a browser and can be accessed from any device with internet connection.")
    
    # Follow-up to web-based
    tree.add_follow_up_question("how can i build one?", 
                               "You'll need to learn HTML, CSS, JavaScript, and a backend language like Python, Node.js, or PHP.")
    
    # Navigate back to parent to add new branch
    tree.navigate_to_question("what is SD?")
    
    # Add desktop branch
    desktop_node = tree.add_follow_up_question("what is an desktop based software?", 
                                              "Desktop software runs locally on your computer and doesn't require internet to function.")
    
    # Add language question
    lang_node = tree.add_follow_up_question("what language I should learn?", 
                                           "For desktop applications, Java or C# are good choices.")
    
    # Add C# question
    tree.add_follow_up_question("what is C#?", 
                               "C# (pronounced C-sharp) is a programming language developed by Microsoft primarily for Windows applications.")
    
    # Print the tree
    print("Question Tree Structure:")
    tree.print_full_tree()
    
    return tree


# To use it with a chatbot, you would do something like:
def process_new_question(question_tree, user_question, chatbot_answer):
    """Process a new question asked by the user and the chatbot's answer"""
    # If tree is empty, this is the first question
    if not question_tree.root:
        question_tree.add_root_question(user_question, chatbot_answer)
    else:
        # Add as a follow-up to the current question
        question_tree.add_follow_up_question(user_question, chatbot_answer)
    
    return question_tree.current_node


# if __name__ == "__main__":
# Demonstrate the conversation tree with your example
demo_tree = demo_conversation_tree()

# This is how you'd use it in a real chatbot application
print("\nSimulating a live conversation with a chatbot:")
live_tree = QuestionTree()

# First question
q1 = "What are common programming paradigms?"
a1 = "Common programming paradigms include object-oriented, functional, procedural, and event-driven programming."
node1 = process_new_question(live_tree, q1, a1)
print(f"Added: {node1}")

# Second question (follow-up)
q2 = "What is functional programming?"
a2 = "Functional programming treats computation as the evaluation of mathematical functions and avoids changing state and mutable data."
node2 = process_new_question(live_tree, q2, a2)
print(f"Added: {node2}")

# Show the live tree
print("\nLive conversation tree:")
live_tree.print_full_tree()

Question Tree Structure:
Q: what is SD?
A: Software Development involves creating different types: web based, android, desktop, etc.
  Q: what is a web based software?
  A: Web-based software runs in a browser and can be accessed from any device with internet connection.
    Q: how can i build one?
    A: You'll need to learn HTML, CSS, JavaScript, and a backend language like Python, Node.js, or PHP.
  Q: what is an desktop based software?
  A: Desktop software runs locally on your computer and doesn't require internet to function.
    Q: what language I should learn?
    A: For desktop applications, Java or C# are good choices.
      Q: what is C#?
      A: C# (pronounced C-sharp) is a programming language developed by Microsoft primarily for Windows applications.

Simulating a live conversation with a chatbot:
Added: Q: What are common programming paradigms?
A: Common programming paradigms include object-oriented, functional, procedural, and event-driven programming.
Added: Q: What i

In [6]:
import os
import cv2
import time
import os

# Set the environment variable to handle authorization on macOS
os.environ["OPENCV_AVFOUNDATION_SKIP_AUTH"] = "0"

def capture_webcam_image_every_5_seconds(save_folder='captured_images'):
    # Create the folder to save images if it doesn't exist
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Open the webcam (0 is the default webcam on most systems)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Could not access the webcam.")
        return

    try:
        while True:
            # Capture a frame from the webcam
            ret, frame = cap.read()

            if not ret:
                print("Error: Failed to capture image.")
                break

            # Get the current timestamp for the image filename
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            image_filename = os.path.join(save_folder, f"{timestamp}.png")

            # Save the captured image
            cv2.imwrite(image_filename, frame)

            print(f"Image saved as {image_filename}")

            # Wait for 5 seconds before capturing the next image
            time.sleep(5)

    except KeyboardInterrupt:
        print("\nCapture stopped by user.")
    finally:
        # Release the webcam and close all OpenCV windows
        cap.release()
        cv2.destroyAllWindows()

# Call the function
capture_webcam_image_every_5_seconds()


Image saved as captured_images/20250226_205142.png
Image saved as captured_images/20250226_205147.png
Image saved as captured_images/20250226_205152.png
Image saved as captured_images/20250226_205157.png
Image saved as captured_images/20250226_205202.png

Capture stopped by user.


In [9]:
import json

class QuestionNode:
    def __init__(self, question, answer=None):
        self.question = question
        self.answer = answer
        self.children = []  # Child questions that follow from this question/answer
        self.parent = None  # Reference to parent question

    def add_child(self, child_node):
        """Add a child question node to this node"""
        self.children.append(child_node)
        child_node.parent = self
        return child_node

    def find_node(self, question):
        """Find a node with the given question text"""
        if self.question == question:
            return self

        for child in self.children:
            result = child.find_node(question)
            if result:
                return result

        return None

    def __str__(self):
        return f"Q: {self.question}\nA: {self.answer}"

    def print_tree(self, indent=0):
        """Print the tree structure starting from this node"""
        print("  " * indent + f"Q: {self.question}")
        if self.answer:
            print("  " * indent + f"A: {self.answer}")

        for child in self.children:
            child.print_tree(indent + 1)

    def to_dict(self):
        """Convert the QuestionNode to a dictionary for JSON serialization"""
        return {
            'question': self.question,
            'answer': self.answer,
            'children': [child.to_dict() for child in self.children]
        }

    @staticmethod
    def from_dict(data):
        """Create a QuestionNode from a dictionary (for JSON deserialization)"""
        node = QuestionNode(data['question'], data['answer'])
        for child_data in data.get('children', []):
            child_node = QuestionNode.from_dict(child_data)
            node.add_child(child_node)
        return node


class QuestionTree:
    def __init__(self):
        self.root = None
        self.current_node = None
        self.context_history = []  # Track the conversation path

    def add_root_question(self, question, answer=None):
        """Add the first question to create the root of the tree"""
        self.root = QuestionNode(question, answer)
        self.current_node = self.root
        self.context_history = [self.root]
        return self.root

    def add_follow_up_question(self, question, answer=None, parent_question=None):
        """Add a follow-up question to a specific parent or to the current question"""
        if not self.root:
            return self.add_root_question(question, answer)

        # If parent specified, find that node
        if parent_question:
            parent_node = self.root.find_node(parent_question)
            if not parent_node:
                raise ValueError(f"Parent question '{parent_question}' not found in tree")
        else:
            # Use current conversation node as parent
            parent_node = self.current_node

        # Create and add new question node
        new_node = QuestionNode(question, answer)
        parent_node.add_child(new_node)

        # Update current node and history
        self.current_node = new_node
        self.context_history.append(new_node)

        return new_node

    def get_current_context(self):
        """Return the current conversation path"""
        return [(node.question, node.answer) for node in self.context_history]

    def navigate_to_question(self, question):
        """Navigate to a specific question in the tree"""
        node = self.root.find_node(question)
        if node:
            self.current_node = node

            # Rebuild context history
            history = []
            current = node
            while current:
                history.insert(0, current)
                current = current.parent

            self.context_history = history
            return True
        return False

    def print_full_tree(self):
        """Print the entire question tree"""
        if self.root:
            self.root.print_tree()
        else:
            print("Tree is empty")

    def generate_dot_graph(self):
        """Generate a DOT representation for visualization with Graphviz"""
        if not self.root:
            return "digraph G {}"

        dot = ["digraph G {"]
        dot.append('  node [shape=box style="rounded,filled" fillcolor=lightblue];')

        # Queue for BFS traversal
        queue = [(self.root, None)]
        node_id = 0
        node_ids = {self.root: node_id}

        while queue:
            node, parent_id = queue.pop(0)
            node_text = f"{node.question}\\n{node.answer if node.answer else ''}"
            node_text = node_text.replace('"', '\\"')

            dot.append(f'  node{node_ids[node]} [label="{node_text}"];')

            if parent_id is not None:
                dot.append(f'  node{parent_id} -> node{node_ids[node]};')

            for child in node.children:
                node_id += 1
                node_ids[child] = node_id
                queue.append((child, node_ids[node]))

        dot.append("}")
        return "\n".join(dot)

    def to_dict(self):
        """Convert the QuestionTree to a dictionary for JSON serialization"""
        if self.root:
            return {'root': self.root.to_dict()}
        return {'root': None}

    @staticmethod
    def from_dict(data):
        """Create a QuestionTree from a dictionary (for JSON deserialization)"""
        tree = QuestionTree()
        root_data = data.get('root')
        if root_data:
            tree.root = QuestionNode.from_dict(root_data)
            tree.current_node = tree.root # set current node to root when loading
            tree.context_history = [tree.root] # reset context history when loading
        return tree


def update_qa_json(question_tree, question, answer, qa_json_file="qa_tree.json"):
    """
    Adds a question and answer to the question tree and updates the qa_json file.

    Args:
        question_tree: The QuestionTree object to update.
        question: The new question to add.
        answer: The answer to the new question.
        qa_json_file: The path to the JSON file to update (default: "qa_tree.json").
    """
    if not question_tree.root:
        question_tree.add_root_question(question, answer)
    else:
        question_tree.add_follow_up_question(question, answer)

    # Serialize the QuestionTree to a dictionary
    tree_dict = question_tree.to_dict()

    # Write the dictionary to the JSON file
    with open(qa_json_file, 'w') as f:
        json.dump(tree_dict, f, indent=4) # indent for pretty printing


def load_qa_json(qa_json_file="qa_tree.json"):
    """
    Loads a QuestionTree from a JSON file.

    Args:
        qa_json_file: The path to the JSON file to load (default: "qa_tree.json").

    Returns:
        A QuestionTree object loaded from the JSON file, or a new empty QuestionTree if the file doesn't exist.
    """
    try:
        with open(qa_json_file, 'r') as f:
            tree_dict = json.load(f)
            return QuestionTree.from_dict(tree_dict)
    except FileNotFoundError:
        return QuestionTree() # Return a new empty tree if file not found


# Example Usage:

# Load existing tree from JSON or create a new one
live_tree = load_qa_json()

# First question
q1 = "What are common programming paradigms?"
a1 = "Common programming paradigms include object-oriented, functional, procedural, and event-driven programming."
update_qa_json(live_tree, q1, a1)
print(f"Added: Q: {q1}, A: {a1}")

# Second question (follow-up)
q2 = "What is functional programming?"
a2 = "Functional programming treats computation as the evaluation of mathematical functions and avoids changing state and mutable data."
update_qa_json(live_tree, q2, a2)
print(f"Added: Q: {q2}, A: {a2}")

# Third question (follow-up)
q3 = "give me an example of functional programming"
a3 = "Haskell and Lisp are examples of functional programming languages."
update_qa_json(live_tree, q3, a3)
print(f"Added: Q: {q3}, A: {a3}")


# Show the live tree in console
print("\nLive conversation tree:")
live_tree.print_full_tree()

# Load the tree back from JSON to verify persistence
loaded_tree = load_qa_json()
print("\nLoaded tree from JSON:")
loaded_tree.print_full_tree()

AttributeError: 'list' object has no attribute 'get'